In [ ]:
import CSV

In [ ]:
using Latexify

In [ ]:
using DataFrames: DataFrame

In [ ]:
ENV["COLUMNS"] = 200

# Read the data

In [ ]:
# get the numbers
df = CSV.read("result.csv", DataFrame);

In [ ]:
df.prec = df.prec .* 100 .|> x->round(x, digits=1);

In [ ]:
df.recall = df.recall .* 100 .|> x->round(x, digits=1);

In [ ]:
df.shd = df.shd .|> x->round(x, digits=1);

# train on ER and test on SF, and vice versa

In [ ]:
df0 = df[occursin.(r"EQ.*gtype=(ER|SF).*mat=CH3", df.expID), :]

In [ ]:
df0.expID

In [ ]:
# Caution: run only once
# df0.expID = map(s->join(match(r"d=(\d+).*gtype=(ER|SF)", s).captures, "-"), df0.expID)

In [ ]:
df0.d = map(s->match(r"d=(\d+).*gtype=(ER|SF)", s)[1], df0.expID)

In [ ]:
df0.train_gtype = map(s->match(r"d=(\d+).*gtype=(ER|SF)", s)[2], df0.expID)

In [ ]:
df0.test_gtype = map(s->match(r"gtype=(ER|SF)", s)[1], df0.testID)

In [ ]:
df1 = sort(df0[!, [:train_gtype, :test_gtype, :d, :prec, :recall, :shd]], [:d, :train_gtype, :test_gtype])

In [ ]:
latexify(df1; env=:table, latex=false)

# Large d on 100, 200, 400

In [ ]:
occursin("a", "ab")

In [ ]:
df[occursin.("d=100", df.expID), :]

In [ ]:
df[occursin.("d=200", df.expID), [:testID, :prec, :recall, :shd]]

In [ ]:
df[occursin.("d=400", df.expID), [:testID, :prec, :recall, :shd]]

In [ ]:
df

# ERSF individual

In [ ]:
df0 = df[df.expID .== "EQ2-ERSF124-d=10-ensemble",[:testID, :prec, :recall, :shd]]

In [ ]:
latexify(df0; env=:table, latex=false)

In [ ]:
df0 = df[df.expID .== "EQ2-ERSF124-d=20-ensemble",[:testID, :prec, :recall, :shd]]

In [ ]:
latexify(df0; env=:table, latex=false)

# Test the ensemble model on unseen d

In [ ]:
df0 = df[df.expID .== "EQ2-CH3-d=[10,15,20]-ensemble",[:testID, :prec, :recall, :shd]]

In [ ]:
latexify(df0; env=:table, latex=false)

In [ ]:
df0 = df[df.expID .== "EQ2-CH3-d=[20,30,40]-ensemble",[:testID, :prec, :recall, :shd]]

In [ ]:
latexify(df0; env=:table, latex=false)

In [ ]:
df0 = df[df.expID .== "EQ2-ERSF-k1-d=20-ensemble",[:testID, :prec, :recall, :shd]]

In [ ]:
latexify(df0; env=:table, latex=false)

# Test on different noise and the tables

In [ ]:
function get_d(s)
    # re = r"d=(\d+)_k=(\d+)_gtype=(.*)_noise=(.*)_mat=([^_]*)(_mec=MLP)?(_raw)?"
    re = r".*d=(\d+)-ensemble"
    m = match(re, s)
    if isnothing(m) @show s end
    d = parse(Int, m.captures[1])
    return d
end

In [ ]:
function get_noise(s)
    # re = r"d=(\d+)_k=(\d+)_gtype=(.*)_noise=(.*)_mat=([^_]*)(_mec=MLP)?(_raw)?"
    re = r".*noise=(.*)"
    m = match(re, s)
    if isnothing(m) @show s end
    return m.captures[1]
end

In [ ]:
# transform this into a new DataFrame
df.d = (df[!,"expID"] .|> get_d)

In [ ]:
df.test_noise = (df[!,:testID] .|> get_noise)

In [ ]:
[:gaussian for i in 1:size(df)[1]]

In [ ]:
df.train_noise = ["Gaussian" for i in 1:size(df)[1]]

In [ ]:
newdf = sort(df[!,All(:d, :train_noise, :test_noise, :prec, :recall, :shd)], [:d, :test_noise])

In [ ]:
latexify(newdf; env=:table, latex=false)